## Get started
1. Install the requirements by running the following in the command line: 

    ```pip install -r requirements.txt```.

2. Clone pretrained models (optional). I you want to fully explore the project, we recommend you clone the models by running the following in the command line:

    ```make models```



## Conformal classifier

you can use `ConformalClassifier` for conformalize a pytorch classifier. It returns tauple of ligits and prediction set

In [ ]:
import torch
import torchvision
from src.models.conformal_classifier import ConformalClassifier
from src.utils.utils import get_CIFAR10_img_transformer
from src.models.cifar10_conv_model import Cifar10ConvModel
from torch.utils.data import DataLoader, random_split

dataset = torchvision.datasets.CIFAR10(train=True, root='../data/processed', download=True, transform=get_CIFAR10_img_transformer())
test_set = torchvision.datasets.CIFAR10(train=False, root='../data/processed', download=True, transform=get_CIFAR10_img_transformer())
calib_size = 1000
train_size = len(dataset)-calib_size
calib_set, train_set = random_split(dataset, [calib_size, train_size])
# load datasets
calib_loader = DataLoader(calib_set)
test_loader = DataLoader(test_set)
# load pretrained model
pretrained_model = Cifar10ConvModel()
model = Cifar10ConvModel()
model.load_state_dict(torch.load(
    "../models/trained_conv_cifar10_model.pt"))
model.eval()
# conformalize the model
conformal_model = ConformalClassifier(model=pretrained_model, calib_loader=calib_loader, alpha=0.1)
# prediction
for x, y in test_loader:
    prediction_set = conformal_model(x)

## Conformal regression

you can use `CQR` for conformalize a pytorch quantile regressor. It returns a prediction interval of upper and lower quantiles.


In [ ]:
import torch
import torchvision
from src.models.CQR import CQR
from src.data.make_housing_dataset import CaliforniaHousing
from src.models.quantile_net import QuantileNet
from torch.utils.data import DataLoader, random_split

calib_set = CaliforniaHousing(split='calib', in_folder='../data/raw', out_folder='../data/processed')
test_set = CaliforniaHousing(split='test', in_folder='../data/raw', out_folder='../data/processed')
# load datasets
calib_loader = DataLoader(calib_set)
test_loader = DataLoader(test_set)
# load pretrained model
pretrained_model = QuantileNet(input_size=8)
pretrained_model.load_state_dict(torch.load(
    f"../models/trained_quantile_net0.1_california_housing.pt"))
pretrained_model.eval()
# conformalize the model
conformal_regressor = CQR(model=pretrained_model, calib_loader=calib_loader, alpha=0.1)
# # prediction
for x, y in test_loader:
    prediction_interval = conformal_regressor(x)

## Active leanring

For active learning, the class `Oracle` can be used by passing the model, training set, test set, and sampling parameters. Abailable strategies:
- `conformal-score:largest-set`
- `least-confidence`
- `entropy-sampler`
- `random-sampler`

 See `src/models/oracle` for more details

In [ ]:
import torchvision
from src.models.oracle import Oracle
from src.models.cifar10_conv_model import Cifar10ConvModel
from src.utils.utils import get_CIFAR10_img_transformer
# load datasets
train_set = torchvision.datasets.CIFAR10(
    train=True, root='../data/processed', transform=get_CIFAR10_img_transformer())
test_set = torchvision.datasets.CIFAR10(
    train=False, root='../data/processed', transform=get_CIFAR10_img_transformer())

model = Cifar10ConvModel()
# number of initial training samples 
n_init_training_labels=1000
oracle = Oracle(model=model, train_set=train_set, test_set=test_set, strategy='conformal-score:largest-set', sample_size=1000, n_init_training_labels=n_init_training_labels)
# start active learning for 1 round
oracle.teach(1)
print(oracle.round_accuracies)